# Mudcard
- **what are other types of non iid data we will encounter?**
    - group structure which we cover today
- **What do you mean by lagged when looking at p?**
    - Are you refering to AR(p)?
    - p is the number of lagged features we use in the model
    - when you have a time series observation, you need to bring it to a format which can be used by ML models
    - you need a feature matrix and a target variable.
    - the target variable is a recent observation
    - the features are observations some dt in the past relative to the target variable
    - this is autoregression, the second learning objective of the previous lecture
- **are we going to cover all time series methods in our homework?**
    - No, only AR and VAR
    - Having said that, all of the time series methods we covered in class can come up in tech interviews
    - In fact, I added all these models to the lecture because it was requested by DSI alumni
- **I'm a bit confused about the matrix shown in this lecture of target variable and feature matrix, what are all the features? Are they not independent features in the non-time series dataset anymore?**
    - No, they are lagged versions of the time series observation
- **pretty clear - i hope i don't have to use time splitting - what to do with a mix of non/time-series features is a bit confusing but I think it's just complicated in general**
    - Knowing who asked this question, I can tell you that you will use time series splitting and use a mix of regular and time series features. :)

# <center> Lecture 11: Group-based and stratified splitting</center>

By the end of this lecture, you will be able to
- describe the motivation and importance of group-based splitting 
- apply various group-based splitting strategies
- apply stratified splitting in a classification problem



# The supervised ML pipeline

**0. Data collection/manipulation**: you might have multiple data sources and/or you might have more data than you need
   - you need to be able to read in datasets from various sources (like csv, excel, SQL, parquet, etc)
   - you need to be able to filter the columns/rows you need for your ML model
   - you need to be able to combine the datasets into one dataframe 

**1. Exploratory Data Analysis (EDA)**: you need to understand your data and verify that it doesn't contain errors
   - do as much EDA as you can!
    
<span style="background-color: #FFFF00">**2. Split the data into different sets**: most often the sets are train, validation, and test (or holdout)</span>
   - practitioners often make errors in this step!
   - you can split the data randomly, based on groups, based on time, or any other non-standard way if necessary to answer your ML question

**3. Preprocess the data**: ML models only work if X and Y are numbers! Some ML models additionally require each feature to have 0 mean and 1 standard deviation (standardized features)
   - often the original features you get contain strings (for example a gender feature would contain 'male', 'female', 'non-binary', 'unknown') which needs to be transformed into numbers
   - often the features are not standardized (e.g., age is between 0 and 100) but it needs to be standardized
    
**4. Choose an evaluation metric**: depends on the priorities of the stakeholders
   - often requires quite a bit of thinking and ethical considerations
     
**5. Choose one or more ML techniques**: it is highly recommended that you try multiple models
   - start with simple models like linear or logistic regression
   - try also more complex models like nearest neighbors, support vector machines, random forest, etc.
    
**6. Tune the hyperparameters of your ML models (aka cross-validation or hyperparameter tuning)**
   - ML techniques have hyperparameters that you need to optimize to achieve best performance
   - for each ML model, decide which parameters to tune and what values to try
   - loop through each parameter combination
       - train one model for each parameter combination
       - evaluate how well the model performs on the validation set
   - take the parameter combo that gives the best validation score
   - evaluate that model on the test set to report how well the model is expected to perform on previously unseen data
    
**7. Interpret your model**: black boxes are often not useful
   - check if your model uses features that make sense (excellent tool for debugging)
   - often model predictions are not enough, you need to be able to explain how the model arrived to a particular prediction (e.g., in health care)

## Recall from Lecture 05

- **the i.i.d. assumption**: the examples in the training set are independently and identically distributed according to $D$
    - every $x_i$ is freshly sampled from $D$ and then labelled by $f$
    - that is, $x_i$ and $y_i$ are picked independently of the other instances
    - $S$ is a window through which the learner gets partial info about $D$ and the labeling function $f$
    - the larger the sample gets, the more likely it is that $D$ and $f$ are accurately reflected
- examples of not iid data:
   - data generated by time-dependent processes
   - data has group structure (samples collected from e.g., different subjects, experiments, measurement devices)
- we will get back to this later in the term 
- if there is any sort of time or group structure in your data, it is likely non-iid
    - **time series data**
        - **values are not independent**
        - stocks price
        - covid19 cases
        - weather data
    - **group structure:**
        - **samples are not identically distributed, $D$ might be different for each group**
        - a person appears multiple times in the dataset (e.g., hospital/doctor visits)
        - data is collected on multiple instruments (e.g., equipment failure prediction)
        - geographical data (e.g., data collected about various cities, counties, states, countries)

        

<font color='lightgray'>By the end of this lecture, you will be able to</font>
- **describe the motivation and importance of group-based splitting** 
- <font color='lightgray'>apply various group-based splitting strategies</font>
- <font color='lightgray'>apply stratified splitting in a classification problem</font>

## Importance of group based splitting
- iid is often assumed
    - most auto-ML tools assume iid because it makes the problem easy to solve
- one of the most common mistakes data science practitioners make is to assume iid when it is not correct
- consequences:
    - information leakage
    - the model performs very well on the test set (low generalization error)
    - when it is deployed, it performs poorly on new groups 
- datasets with group structure have some sort of group ID
    - this can be customer ID, patient ID, instrument ID, sensor ID, etc.
    - the group ID should NOT be used as a feature
    - a unique identifier (often just a random sequence of characters) is not something an ML model can use
    - it should be separated out and used as a group ID in group-based splitting methods (more on this later)
- a categorical/ordinal feature is not usually a group ID! 
    - a group ID is usually a column that does not contain info the ML model can use
    - the model can learn from categories 


# An example: seizure project
- you can read the publication [here](https://ieeexplore.ieee.org/document/8857552)
- classification problem:
   - epileptic seizures vs. non-epileptic psychogenic seizures
- data from empatica wrist sensor
   - heart rate, skin temperature, EDA, blood volume pressure, acceleration
- data collection:
   - patients come to the hospital for a few days
   - eeg and video recording to determine seizure type
   - wrist sensor data is collected
- question:
   - Can we use the wrist sensor data to differentiate the two seizure types on new patients?

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/seizure_data.csv')
print(df[df['patient ID'] == 32])

    patient ID            seizure_ID  ACC_mean  BVP_mean  EDA_mean    HR_mean  \
5           32  ID32__day3_arm_1_sz1  1.028539 -0.092102  0.112795  64.748167   
6           32  ID32__day3_arm_1_sz1  1.027986  0.745437  0.130486  63.715667   
7           32  ID32__day2_arm_1_sz0  1.002146  0.150810  0.189272  61.838500   
8           32  ID32__day2_arm_1_sz0  1.005410  0.482859  1.226038  66.240833   
9           32  ID32__day1_arm_1_sz0  0.997017 -0.925122  0.200990  56.103667   
10          32  ID32__day1_arm_1_sz0  1.009207  1.618456  1.679754  64.668167   
27          32  ID32__day1_arm_1_sz0  1.000290  0.046690  0.123165  54.289500   
28          32  ID32__day1_arm_1_sz0  1.010351  0.125039  0.471180  65.060667   
29          32  ID32__day2_arm_1_sz0  1.018163  0.254302  0.206010  61.875833   
30          32  ID32__day2_arm_1_sz0  1.016785  1.242893  0.954649  66.216167   
34          32  ID32__day3_arm_1_sz1  1.008867  0.070180  0.195966  65.995667   
35          32  ID32__day3_a

In [2]:
y = df['label']
patient_ID = df['patient ID']
seizure_ID = df['seizure_ID']
X = df.drop(columns=['patient ID','seizure_ID','label'])
classes, counts = np.unique(y,return_counts=True)
print(classes, counts)
print('balance:',np.max(counts/len(y)))

[0. 1.] [ 86 190]
balance: 0.6884057971014492


In [3]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

def ML_pipeline_kfold_GridSearchCV(X,y,random_state,n_folds):
    # create a test set
    X_other, X_test, y_other, y_test = train_test_split(X, y, test_size=0.2, random_state = random_state)
    # splitter for _other
    kf = StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=random_state)
    # create the pipeline: preprocessor + supervised ML method
    scaler = StandardScaler()
    pipe = make_pipeline(scaler,SVC())
    # the parameter(s) we want to tune
    param_grid = {'svc__C': np.logspace(-3,4,num=8),'svc__gamma': np.logspace(-3,4,num=8)}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                        cv=kf, return_train_score = True)
    # do kfold CV on _other
    grid.fit(X_other, y_other)
    return grid, grid.score(X_test, y_test)

In [4]:
test_scores = []
for i in range(5):
    grid, test_score = ML_pipeline_kfold_GridSearchCV(X,y,i*42,5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

{'svc__C': np.float64(100.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.9363636363636363
test score: 0.8214285714285714
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.01)}
best CV score: 0.9136363636363635
test score: 0.9285714285714286
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.01)}
best CV score: 0.9227272727272726
test score: 0.9464285714285714
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.01)}
best CV score: 0.9318181818181819
test score: 0.8928571428571429
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.9272727272727274
test score: 0.875
test accuracy: 0.89 +/- 0.04


## This is wrong! A very bad case of data leakage!
- the textbook case of data/information leakage!
- if we just do KFold CV blindly, the points from the same patient end up in different sets
   - when you deploy the model and apply it to data from new patients, that patient's data will be seen for the first time
- the ML pipeline needs to mimic the intended use of the model!
   - we want to split the points based on the patient ID!
   - we want all points from the same patient to be in either train/CV/test

## Group-based split: GroupKFold
<center><img src="../figures/groupkfold.png" width="600"></center>


In [5]:
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
def ML_pipeline_groups_GridSearchCV(X,y,groups,random_state,n_folds):
    # create a test set based on groups
    splitter = GroupShuffleSplit(n_splits=1,test_size=0.2,random_state=random_state)
    for i_other,i_test in splitter.split(X, y, groups):
        X_other, y_other, groups_other = X.iloc[i_other], y.iloc[i_other], groups.iloc[i_other]
        X_test, y_test, groups_test = X.iloc[i_test], y.iloc[i_test], groups.iloc[i_test]
    # check the split
#     print(pd.unique(groups))
#     print(pd.unique(groups_other))
#     print(pd.unique(groups_test))
    # splitter for _other
    kf = GroupKFold(n_splits=n_folds)
    # create the pipeline: preprocessor + supervised ML method
    scaler = StandardScaler()
    pipe = make_pipeline(scaler,SVC())
    # the parameter(s) we want to tune
    param_grid = {'svc__C': np.logspace(-3,4,num=8),'svc__gamma': np.logspace(-3,4,num=8)}
    # prepare gridsearch
    grid = GridSearchCV(pipe, param_grid=param_grid,scoring = make_scorer(accuracy_score),
                        cv=kf, return_train_score = True)
    # do kfold CV on _other
    grid.fit(X_other, y_other, groups=groups_other)
    return grid, grid.score(X_test, y_test)

In [6]:
test_scores = []
for i in range(5):
    grid, test_score = ML_pipeline_groups_GridSearchCV(X,y,patient_ID,i*42,5)
    print(grid.best_params_)
    print('best CV score:',grid.best_score_)
    print('test score:',test_score)
    test_scores.append(test_score)
print('test accuracy:',np.around(np.mean(test_scores),2),'+/-',np.around(np.std(test_scores),2))

{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.7609139784946237
test score: 0.6410256410256411
{'svc__C': np.float64(0.1), 'svc__gamma': np.float64(0.01)}
best CV score: 0.6522727272727272
test score: 0.2711864406779661
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.5720073891625616
test score: 0.9390243902439024
{'svc__C': np.float64(10.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.7061742424242425
test score: 0.43243243243243246
{'svc__C': np.float64(10000.0), 'svc__gamma': np.float64(0.001)}
best CV score: 0.6082407407407406
test score: 0.8901098901098901
test accuracy: 0.63 +/- 0.26


## The takeaway
- an incorrect cross validation pipeline gives misleading results
   - usually the model appears to be pretty accurate
   - but the performance is poor when the model is deployed
- this can be avoided by a careful cross validation pipeline
   - think about how your model will be used
   - mimic that future use in CV

<font color='lightgray'>By the end of this lecture, you will be able to</font>
- <font color='lightgray'>describe the motivation and importance of group-based splitting</font>
- **apply various group-based splitting strategies**
- <font color='lightgray'>apply stratified splitting in a classification problem</font>

# Let's take a look at group splitters using toy datasets

## Group-based split: GroupKFold
<center><img src="../figures/groupkfold.png" width="600"></center>


In [17]:
from sklearn.model_selection import GroupKFold
import numpy as np

X = np.ones(shape=(8, 2))
y = np.ones(shape=(8, 1))
groups = np.array([1, 1, 2, 2, 2, 3, 3, 3])

group_kfold = GroupKFold(n_splits=3)

for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)


TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [0 1 5 6 7] TEST: [2 3 4]
TRAIN: [2 3 4 5 6 7] TEST: [0 1]


In [8]:
help(GroupKFold)

Help on class GroupKFold in module sklearn.model_selection._split:

class GroupKFold(GroupsConsumerMixin, _BaseKFold)
 |  GroupKFold(n_splits=5, *, shuffle=False, random_state=None)
 |
 |  K-fold iterator variant with non-overlapping groups.
 |
 |  Each group will appear exactly once in the test set across all folds (the
 |  number of distinct groups has to be at least equal to the number of folds).
 |
 |  The folds are approximately balanced in the sense that the number of
 |  samples is approximately the same in each test fold when `shuffle` is True.
 |
 |  Read more in the :ref:`User Guide <group_k_fold>`.
 |
 |  For visualisation of cross-validation behaviour and
 |  comparison between common scikit-learn split methods
 |  refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`
 |
 |  Parameters
 |  ----------
 |  n_splits : int, default=5
 |      Number of folds. Must be at least 2.
 |
 |      .. versionchanged:: 0.22
 |          ``n_splits`` default value change

## Group-based split: GroupShuffleSplit
<center><img src="../figures/groupshufflesplit.png" width="600"></center>


In [9]:
from sklearn.model_selection import GroupShuffleSplit

gss = GroupShuffleSplit(n_splits=10, train_size=.8, random_state=0)

for train_idx, test_idx in gss.split(X, y, groups):
    print("TRAIN:", train_idx, "TEST:", test_idx)
    

TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [2 3 4 5 6 7] TEST: [0 1]
TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [0 1 2 3 4] TEST: [5 6 7]
TRAIN: [0 1 5 6 7] TEST: [2 3 4]
TRAIN: [0 1 5 6 7] TEST: [2 3 4]
TRAIN: [2 3 4 5 6 7] TEST: [0 1]
TRAIN: [2 3 4 5 6 7] TEST: [0 1]
TRAIN: [0 1 5 6 7] TEST: [2 3 4]


## Quiz 1
Go back to the GroupKFold example above. What happens when you change n_splits to 4? Why?

Why could we set the n_splits argument to 10 in GroupShuffleSplit? Check the manual of both methods to find the answer.

Explain your answer in a couple of sentences!

<font color='lightgray'>By the end of this lecture, you will be able to</font>
- <font color='lightgray'>describe the motivation and importance of group-based splitting</font>
- <font color='lightgray'>apply various group-based splitting strategies</font>
- **apply stratified splitting in a classification problem**

## Imbalanced data
- imbalanced data: only a small fraction of the points are in one of the classes, usually ~5% or less but there is no hard limit here
- examples:
    - people visit a bank's website. do they sign up for a new credit card?
        - most customers just browse and leave the page
        - usually 1% or less of the customers get a credit card (class 1), the rest leaves the page without signing up (class 0).
    - fraud detection
        - only a tiny fraction of credit card payments are fraudulent
    - rare disease diagnosis
- the issue with imbalanced data:
    - if you apply train_test_split or KFold, you might not have class 1 points in one of your sets by chance
    - this is what we need to fix

## Solution: stratified splits

In [10]:
df = pd.read_csv('../data/imbalanced_data.csv')

X = df[['feature1','feature2']]
y = df['y']

print(y.value_counts())


y
0    990
1     10
Name: count, dtype: int64


In [11]:
# 4 and 10

random_state = 4

X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,random_state=random_state)

print('**balance without stratification:**')
# a variation on the order of 1% which would be too much for imbalanced data!
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))

X_train, X_other, y_train, y_other = train_test_split(X,y,train_size = 0.6,stratify=y,random_state=random_state)
X_val, X_test, y_val, y_test = train_test_split(X_other,y_other,train_size = 0.5,stratify=y_other,random_state=random_state)
print('**balance with stratification:**')
# very little variation (in the 4th decimal point only) which is important if the problem is imbalanced
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))


**balance without stratification:**
(array([0, 1]), array([591,   9]))
(array([0]), array([200]))
(array([0, 1]), array([199,   1]))
**balance with stratification:**
(array([0, 1]), array([594,   6]))
(array([0, 1]), array([198,   2]))
(array([0, 1]), array([198,   2]))


## Stratified folds
<center><img src="../figures/stratified_kfold.png" width="600"></center>


In [12]:
from sklearn.model_selection import StratifiedKFold
help(StratifiedKFold)

Help on class StratifiedKFold in module sklearn.model_selection._split:

class StratifiedKFold(_BaseKFold)
 |  StratifiedKFold(n_splits=5, *, shuffle=False, random_state=None)
 |
 |  Stratified K-Fold cross-validator.
 |
 |  Provides train/test indices to split data in train/test sets.
 |
 |  This cross-validation object is a variation of KFold that returns
 |  stratified folds. The folds are made by preserving the percentage of
 |  samples for each class.
 |
 |  Read more in the :ref:`User Guide <stratified_k_fold>`.
 |
 |  For visualisation of cross-validation behaviour and
 |  comparison between common scikit-learn split methods
 |  refer to :ref:`sphx_glr_auto_examples_model_selection_plot_cv_indices.py`
 |
 |  Parameters
 |  ----------
 |  n_splits : int, default=5
 |      Number of folds. Must be at least 2.
 |
 |      .. versionchanged:: 0.22
 |          ``n_splits`` default value changed from 3 to 5.
 |
 |  shuffle : bool, default=False
 |      Whether to shuffle each class's s

In [13]:
# what we did before: variance in balance on the order of 1%
random_state = 2

X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,random_state=random_state)
print('test balance:',np.unique(y_test,return_counts=True))

# do KFold split on other
kf = KFold(n_splits=4,shuffle=True,random_state=random_state)
for train_index, val_index in kf.split(X_other,y_other):
    print('new fold')
    X_train = X_other.iloc[train_index]
    y_train = y_other.iloc[train_index]
    X_val = X_other.iloc[val_index]
    y_val = y_other.iloc[val_index]
    print(np.unique(y_train,return_counts=True))
    print(np.unique(y_val,return_counts=True))

test balance: (array([0, 1]), array([198,   2]))
new fold
(array([0, 1]), array([596,   4]))
(array([0, 1]), array([196,   4]))
new fold
(array([0, 1]), array([593,   7]))
(array([0, 1]), array([199,   1]))
new fold
(array([0, 1]), array([592,   8]))
(array([0]), array([200]))
new fold
(array([0, 1]), array([595,   5]))
(array([0, 1]), array([197,   3]))


In [14]:
# stratified K Fold: variation in balance is very small (4th decimal point)
random_state = 42

# stratified train-test split
X_other, X_test, y_other, y_test = train_test_split(X,y,test_size = 0.2,stratify=y,random_state=random_state)
print('test balance:',np.unique(y_test,return_counts=True))

# do StratifiedKFold split on other
kf = StratifiedKFold(n_splits=4,shuffle=True,random_state=random_state)
for train_index, val_index in kf.split(X_other,y_other):
    print('new fold')
    X_train = X_other.iloc[train_index]
    y_train = y_other.iloc[train_index]
    X_val = X_other.iloc[val_index]
    y_val = y_other.iloc[val_index]
    print(np.unique(y_train,return_counts=True))
    print(np.unique(y_val,return_counts=True))

test balance: (array([0, 1]), array([198,   2]))
new fold
(array([0, 1]), array([594,   6]))
(array([0, 1]), array([198,   2]))
new fold
(array([0, 1]), array([594,   6]))
(array([0, 1]), array([198,   2]))
new fold
(array([0, 1]), array([594,   6]))
(array([0, 1]), array([198,   2]))
new fold
(array([0, 1]), array([594,   6]))
(array([0, 1]), array([198,   2]))


## Mudcard